# Análisis de Parámetros del Algoritmo Ant Colony System (ACS)

Este notebook proporciona un análisis completo de los parámetros del algoritmo ACS para el problema del viajante de comercio (TSP Berlin52). 

## Información del Problema:
- **Instancia**: Berlin52 (52 ciudades)
- **Óptimo conocido**: 7544.3659 (verificado por múltiples fuentes)
- **Objetivo**: Encontrar los mejores parámetros para el algoritmo ACS

## Objetivos del Análisis:
1. Validar la correctitud de los tests de parámetros
2. Realizar análisis estadístico robusto de los resultados
3. Identificar los mejores parámetros para el algoritmo
4. Proporcionar visualizaciones comprensivas del rendimiento

## Metodología:
- Múltiples ejecuciones independientes para cada configuración de parámetros
- Análisis estadístico con intervalos de confianza
- Visualizaciones para identificar patrones y tendencias
- Recomendaciones basadas en evidencia estadística
- Cálculo de gap porcentual basado en el óptimo conocido (7544.3659)

## 1. Import Required Libraries

In [ ]:
# Librerías básicas para análisis de datos
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Librerías para análisis estadístico
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Configuración de pandas para mostrar más columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Librerías importadas correctamente")
print("📊 Configuración de visualización establecida")

In [ ]:
# Configuración específica del problema
BERLIN52_OPTIMAL_COST = 7544.3659  # Valor óptimo verificado para Berlin52
TSP_FILE = "./berlin52.tsp"

print(f"🎯 Problema: Berlin52 TSP")
print(f"📊 Valor óptimo conocido: {BERLIN52_OPTIMAL_COST}")
print(f"📁 Archivo TSP: {TSP_FILE}")
print(f"🔍 Todos los análisis de gap se basarán en este valor óptimo verificado")

## 2. Load and Prepare Test Data

In [ ]:
# Función para cargar datos de análisis de parámetros
def load_parameter_analysis_data(file_path=None, recalculate_gap=True):
    """
    Carga los datos del análisis de parámetros desde un archivo JSON.
    Si no se especifica archivo, busca el más reciente.
    
    Args:
        file_path: Ruta al archivo JSON (opcional)
        recalculate_gap: Si True, recalcula el gap con el valor óptimo correcto
    """
    if file_path is None:
        # Buscar el archivo más reciente
        current_dir = Path('.')
        json_files = list(current_dir.glob('*parameter_analysis_*.json')) + \
                    list(current_dir.glob('*statistical_tsp_results_*.json'))
        if not json_files:
            print("❌ No se encontraron archivos de análisis.")
            print("💡 Ejecuta primero uno de estos scripts:")
            print("   - parameter_analysis_test.py")
            print("   - improved_statistical_test.py")
            return None
        file_path = max(json_files, key=lambda x: x.stat().st_mtime)
        print(f"📂 Cargando archivo más reciente: {file_path}")
    
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        df = pd.DataFrame(data)
        
        # Recalcular gap percentage con el valor óptimo correcto si es necesario
        if recalculate_gap:
            df['gap_percentage'] = ((df['final_cost'] - BERLIN52_OPTIMAL_COST) / BERLIN52_OPTIMAL_COST) * 100
            print(f"🔄 Gap percentage recalculado con óptimo correcto: {BERLIN52_OPTIMAL_COST}")
        
        print(f"✅ Datos cargados: {len(df)} experimentos")
        return df
    
    except FileNotFoundError:
        print(f"❌ Archivo no encontrado: {file_path}")
        return None
    except Exception as e:
        print(f"❌ Error al cargar datos: {e}")
        return None

# Cargar datos
df = load_parameter_analysis_data()

if df is not None:
    print(f"\n📊 Información del dataset:")
    print(f"   - Forma: {df.shape}")
    print(f"   - Parámetros analizados: {df[['colony_size', 'alpha', 'beta', 'q0']].nunique().to_dict()}")
    print(f"   - Rango de costos: {df['final_cost'].min():.2f} - {df['final_cost'].max():.2f}")
    print(f"   - Mejor costo vs óptimo: {df['final_cost'].min():.2f} vs {BERLIN52_OPTIMAL_COST}")
    print(f"   - Rango de gaps: {df['gap_percentage'].min():.2f}% - {df['gap_percentage'].max():.2f}%")
    
    # Verificar si alguna corrida alcanzó el óptimo
    optimal_reached = (df['final_cost'] <= BERLIN52_OPTIMAL_COST + 0.01).sum()
    if optimal_reached > 0:
        print(f"   🎯 Corridas que alcanzaron el óptimo (±0.01): {optimal_reached}")
    
    # Mostrar primeras filas
    print(f"\n📋 Primeras filas del dataset:")
    display(df.head())
else:
    print("\n⚠️  No se pudieron cargar los datos.")
    print("Ejecuta uno de estos comandos para generar datos:")
    print("python parameter_analysis_test.py")
    print("python improved_statistical_test.py")

## 3. Validate Test Correctness

Antes de proceder con el análisis, validemos que la metodología de testing es correcta.

In [ ]:
def validate_test_methodology(df):
    """
    Valida la correctitud de la metodología de testing
    """
    print("🔍 VALIDACIÓN DE LA METODOLOGÍA DE TESTING")
    print("=" * 50)
    
    # 1. Verificar múltiples ejecuciones
    runs_per_config = df.groupby(['colony_size', 'alpha', 'beta', 'q0']).size()
    min_runs = runs_per_config.min()
    max_runs = runs_per_config.max()
    avg_runs = runs_per_config.mean()
    
    print(f"1. 📊 MÚLTIPLES EJECUCIONES:")
    print(f"   - Mínimo de corridas: {min_runs}")
    print(f"   - Máximo de corridas: {max_runs}")
    print(f"   - Promedio de corridas: {avg_runs:.1f}")
    
    if min_runs >= 3:
        print("   ✅ BIEN: Múltiples ejecuciones por configuración")
    else:
        print("   ❌ PROBLEMA: Pocas ejecuciones, resultados no confiables")
    
    # 2. Verificar variabilidad en los resultados
    print(f"\n2. 📈 VARIABILIDAD DE RESULTADOS:")
    variability_stats = df.groupby(['colony_size', 'alpha', 'beta', 'q0'])['final_cost'].agg(['std', 'mean'])
    variability_stats['cv'] = variability_stats['std'] / variability_stats['mean']
    avg_cv = variability_stats['cv'].mean()
    
    print(f"   - Coeficiente de variación promedio: {avg_cv:.3f}")
    
    if avg_cv > 0.001:
        print("   ✅ BIEN: Hay variabilidad suficiente en los resultados")
    else:
        print("   ⚠️  ADVERTENCIA: Poca variabilidad, posible problema de semillas")
    
    # 3. Verificar distribución de parámetros
    print(f"\n3. ⚖️  DISTRIBUCIÓN DE PARÁMETROS:")
    for param in ['colony_size', 'alpha', 'beta', 'q0']:
        unique_values = df[param].nunique()
        print(f"   - {param}: {unique_values} valores únicos")
    
    print("   ✅ BIEN: Se probaron múltiples valores de cada parámetro")
    
    # 4. Verificar semillas diferentes
    print(f"\n4. 🎲 VERIFICACIÓN DE SEMILLAS:")
    unique_seeds = df['seed'].nunique()
    total_runs = len(df)
    
    print(f"   - Semillas únicas: {unique_seeds}")
    print(f"   - Total de ejecuciones: {total_runs}")
    
    if unique_seeds == total_runs:
        print("   ✅ EXCELENTE: Cada ejecución usa una semilla diferente")
    elif unique_seeds >= total_runs * 0.8:
        print("   ✅ BIEN: La mayoría de ejecuciones usan semillas diferentes")
    else:
        print("   ⚠️  ADVERTENCIA: Muchas ejecuciones comparten semillas")
    
    # 5. Verificar rango de resultados
    print(f"\n5. 🎯 RANGO DE RESULTADOS:")
    cost_range = df['final_cost'].max() - df['final_cost'].min()
    gap_range = df['gap_percentage'].max() - df['gap_percentage'].min()
    
    print(f"   - Rango de costos: {cost_range:.2f}")
    print(f"   - Rango de gaps: {gap_range:.2f}%")
    
    if gap_range > 5:
        print("   ✅ BIEN: Amplio rango de resultados permite discriminar parámetros")
    else:
        print("   ⚠️  ADVERTENCIA: Rango estrecho, difícil discriminar parámetros")
    
    return {
        'multiple_runs': min_runs >= 3,
        'sufficient_variability': avg_cv > 0.001,
        'unique_seeds': unique_seeds >= total_runs * 0.8,
        'good_range': gap_range > 5
    }

# Ejecutar validación si tenemos datos
if df is not None:
    validation_results = validate_test_methodology(df)
    
    print(f"\n🏆 RESUMEN DE VALIDACIÓN:")
    all_good = all(validation_results.values())
    if all_good:
        print("✅ METODOLOGÍA EXCELENTE: Todos los criterios se cumplen")
    else:
        print("⚠️  METODOLOGÍA MEJORABLE: Algunos criterios no se cumplen completamente")
        print("💡 Considera ejecutar más corridas para mejorar la confiabilidad")
else:
    print("⚠️  No hay datos para validar. Ejecuta primero el análisis de parámetros.")

## 4. Statistical Analysis of Results

In [ ]:
def calculate_statistics_by_parameter(df, confidence_level=0.95):
    """
    Calcula estadísticas detalladas agrupadas por cada parámetro
    """
    if df is None:
        return None
    
    # Configurar el nivel de confianza
    alpha = 1 - confidence_level
    
    results = {}
    
    for param in ['colony_size', 'alpha', 'beta', 'q0']:
        print(f"\n📊 ANÁLISIS ESTADÍSTICO: {param.upper()}")
        print("=" * 50)
        
        # Agrupar por el parámetro
        grouped = df.groupby(param)
        
        stats_list = []
        for value, group in grouped:
            n = len(group)
            mean_cost = group['final_cost'].mean()
            std_cost = group['final_cost'].std()
            
            # Intervalo de confianza para la media
            sem = std_cost / np.sqrt(n)  # Error estándar de la media
            t_value = stats.t.ppf(1 - alpha/2, df=n-1)
            ci_lower = mean_cost - t_value * sem
            ci_upper = mean_cost + t_value * sem
            
            # Gap percentage
            mean_gap = group['gap_percentage'].mean()
            std_gap = group['gap_percentage'].std()
            
            # Tiempo de ejecución
            mean_time = group['execution_time'].mean()
            
            stats_dict = {
                param: value,
                'n_runs': n,
                'mean_cost': mean_cost,
                'std_cost': std_cost,
                'ci_lower': ci_lower,
                'ci_upper': ci_upper,
                'mean_gap': mean_gap,
                'std_gap': std_gap,
                'mean_time': mean_time,
                'best_cost': group['final_cost'].min(),
                'worst_cost': group['final_cost'].max()
            }
            stats_list.append(stats_dict)
        
        # Crear DataFrame con estadísticas
        param_stats = pd.DataFrame(stats_list)
        param_stats = param_stats.sort_values('mean_cost')
        
        results[param] = param_stats
        
        # Mostrar tabla
        display_cols = [param, 'n_runs', 'mean_cost', 'ci_lower', 'ci_upper', 'mean_gap', 'best_cost']
        print(param_stats[display_cols].round(2).to_string(index=False))
        
        # Identificar el mejor valor
        best_idx = param_stats['mean_cost'].idxmin()
        best_value = param_stats.loc[best_idx, param]
        best_cost = param_stats.loc[best_idx, 'mean_cost']
        best_gap = param_stats.loc[best_idx, 'mean_gap']
        
        print(f"\n🏆 MEJOR VALOR: {param} = {best_value}")
        print(f"   - Costo promedio: {best_cost:.2f}")
        print(f"   - Gap promedio: {best_gap:.2f}%")
        
        # Test ANOVA para verificar diferencias significativas
        groups = [group['final_cost'].values for _, group in grouped]
        f_stat, p_value = stats.f_oneway(*groups)
        
        print(f"\n🧮 TEST ANOVA:")
        print(f"   - F-statistic: {f_stat:.3f}")
        print(f"   - p-value: {p_value:.6f}")
        
        if p_value < 0.05:
            print(f"   ✅ Diferencias SIGNIFICATIVAS entre valores de {param}")
        else:
            print(f"   ❌ NO hay diferencias significativas entre valores de {param}")
    
    return results

# Ejecutar análisis estadístico
if df is not None:
    param_statistics = calculate_statistics_by_parameter(df)
else:
    print("⚠️  No hay datos para el análisis estadístico.")

## 5. Visualize Parameter Performance

In [ ]:
def create_parameter_visualizations(df):
    """
    Crea visualizaciones comprehensivas para el análisis de parámetros
    """
    if df is None:
        print("⚠️  No hay datos para visualizar.")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    fig.suptitle('Análisis de Parámetros del Algoritmo ACS', fontsize=16, fontweight='bold')
    
    parameters = ['colony_size', 'alpha', 'beta', 'q0']
    
    for i, param in enumerate(parameters):
        ax = axes[i//2, i%2]
        
        # Box plot con intervalos de confianza
        sns.boxplot(data=df, x=param, y='final_cost', ax=ax)
        ax.set_title(f'Distribución de Costos por {param}', fontweight='bold')
        ax.set_ylabel('Costo Final')
        ax.grid(True, alpha=0.3)
        
        # Rotar etiquetas si es necesario
        if param in ['alpha', 'beta', 'q0']:
            ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Gráfico de barras con intervalos de confianza
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    fig.suptitle('Costo Promedio con Intervalos de Confianza (95%)', fontsize=16, fontweight='bold')
    
    for i, param in enumerate(parameters):
        ax = axes[i//2, i%2]
        
        # Calcular estadísticas por parámetro
        stats_df = df.groupby(param).agg({
            'final_cost': ['mean', 'std', 'count']
        }).reset_index()
        
        stats_df.columns = [param, 'mean', 'std', 'count']
        
        # Calcular intervalos de confianza
        stats_df['sem'] = stats_df['std'] / np.sqrt(stats_df['count'])
        stats_df['ci'] = 1.96 * stats_df['sem']  # 95% CI
        
        # Gráfico de barras
        bars = ax.bar(range(len(stats_df)), stats_df['mean'], 
                     yerr=stats_df['ci'], capsize=5, alpha=0.7)
        
        ax.set_xlabel(param)
        ax.set_ylabel('Costo Promedio')
        ax.set_title(f'Costo Promedio por {param}')
        ax.set_xticks(range(len(stats_df)))
        ax.set_xticklabels(stats_df[param], rotation=45 if param in ['alpha', 'beta', 'q0'] else 0)
        ax.grid(True, alpha=0.3)
        
        # Destacar el mejor valor
        best_idx = stats_df['mean'].idxmin()
        bars[best_idx].set_color('red')
        bars[best_idx].set_alpha(0.9)
    
    plt.tight_layout()
    plt.show()

def create_correlation_analysis(df):
    """
    Análisis de correlación entre parámetros y rendimiento
    """
    if df is None:
        return
    
    print("🔗 ANÁLISIS DE CORRELACIÓN")
    print("=" * 50)
    
    # Matriz de correlación
    corr_columns = ['colony_size', 'alpha', 'beta', 'q0', 'final_cost', 'gap_percentage', 'execution_time']
    corr_matrix = df[corr_columns].corr()
    
    # Visualizar matriz de correlación
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": .8})
    plt.title('Matriz de Correlación entre Parámetros y Métricas de Rendimiento', fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Análisis de correlación con el costo final
    print("🎯 CORRELACIÓN CON EL COSTO FINAL:")
    for param in ['colony_size', 'alpha', 'beta', 'q0']:
        correlation, p_value = pearsonr(df[param], df['final_cost'])
        print(f"   - {param}: r = {correlation:.3f}, p = {p_value:.6f}")
        
        if abs(correlation) > 0.3:
            direction = "positiva" if correlation > 0 else "negativa"
            strength = "fuerte" if abs(correlation) > 0.7 else "moderada"
            print(f"     ➤ Correlación {direction} {strength}")
        else:
            print(f"     ➤ Correlación débil")

def create_performance_summary(df):
    """
    Crea un resumen visual del rendimiento general
    """
    if df is None:
        return
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. Distribución de costos finales
    axes[0].hist(df['final_cost'], bins=30, alpha=0.7, edgecolor='black')
    axes[0].axvline(df['final_cost'].mean(), color='red', linestyle='--', 
                   label=f'Media: {df["final_cost"].mean():.2f}')
    axes[0].axvline(df['final_cost'].median(), color='orange', linestyle='--', 
                   label=f'Mediana: {df["final_cost"].median():.2f}')
    axes[0].set_xlabel('Costo Final')
    axes[0].set_ylabel('Frecuencia')
    axes[0].set_title('Distribución de Costos Finales')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 2. Gap percentage vs execution time
    scatter = axes[1].scatter(df['execution_time'], df['gap_percentage'], 
                             c=df['final_cost'], cmap='viridis', alpha=0.6)
    axes[1].set_xlabel('Tiempo de Ejecución (s)')
    axes[1].set_ylabel('Gap Percentage (%)')
    axes[1].set_title('Tiempo vs Calidad de Solución')
    axes[1].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[1], label='Costo Final')
    
    # 3. Box plot del gap percentage
    axes[2].boxplot(df['gap_percentage'])
    axes[2].set_ylabel('Gap Percentage (%)')
    axes[2].set_title('Distribución del Gap Percentage')
    axes[2].grid(True, alpha=0.3)
    
    plt.suptitle('Resumen de Rendimiento del Algoritmo ACS', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Crear visualizaciones
if df is not None:
    print("📊 Generando visualizaciones...")
    create_parameter_visualizations(df)
    create_correlation_analysis(df)
    create_performance_summary(df)
else:
    print("⚠️  No hay datos para crear visualizaciones.")

## 6. Generate Performance Reports and Best Parameter Recommendations

In [ ]:
def generate_best_parameters_report(df, param_statistics):
    """
    Genera un reporte completo con las mejores configuraciones de parámetros
    """
    if df is None or param_statistics is None:
        print("⚠️  No hay datos suficientes para generar el reporte.")
        return
    
    print("🏆 REPORTE DE MEJORES PARÁMETROS")
    print("=" * 60)
    
    # 1. Mejores valores individuales por parámetro
    best_params = {}
    
    print("\n1. 🎯 MEJORES VALORES INDIVIDUALES:")
    for param in ['colony_size', 'alpha', 'beta', 'q0']:
        stats = param_statistics[param]
        best_idx = stats['mean_cost'].idxmin()
        best_value = stats.loc[best_idx, param]
        best_cost = stats.loc[best_idx, 'mean_cost']
        best_gap = stats.loc[best_idx, 'mean_gap']
        
        best_params[param] = best_value
        
        print(f"   • {param}: {best_value}")
        print(f"     - Costo promedio: {best_cost:.2f}")
        print(f"     - Gap promedio: {best_gap:.2f}%")
    
    # 2. Mejor combinación general
    print(f"\n2. 🌟 MEJOR COMBINACIÓN GENERAL:")
    
    # Encontrar la mejor ejecución individual
    best_run_idx = df['final_cost'].idxmin()
    best_run = df.loc[best_run_idx]
    
    print(f"   • colony_size: {best_run['colony_size']}")
    print(f"   • alpha: {best_run['alpha']}")
    print(f"   • beta: {best_run['beta']}")
    print(f"   • q0: {best_run['q0']}")
    print(f"   • Costo obtenido: {best_run['final_cost']:.2f}")
    print(f"   • Gap: {best_run['gap_percentage']:.2f}%")
    print(f"   • Tiempo: {best_run['execution_time']:.2f}s")
    
    # 3. Top 5 configuraciones
    print(f"\n3. 🥇 TOP 5 MEJORES CONFIGURACIONES:")
    
    # Agrupar por configuración y obtener estadísticas
    config_stats = df.groupby(['colony_size', 'alpha', 'beta', 'q0']).agg({
        'final_cost': ['mean', 'std', 'min'],
        'gap_percentage': 'mean',
        'execution_time': 'mean'
    }).round(2)
    
    config_stats.columns = ['mean_cost', 'std_cost', 'best_cost', 'mean_gap', 'mean_time']
    config_stats = config_stats.sort_values('mean_cost').head(5).reset_index()
    
    for i, row in config_stats.iterrows():
        print(f"   {i+1}. colony_size={row['colony_size']}, alpha={row['alpha']}, "
              f"beta={row['beta']}, q0={row['q0']}")
        print(f"      Costo: {row['mean_cost']:.2f} ± {row['std_cost']:.2f}, "
              f"Gap: {row['mean_gap']:.2f}%, Tiempo: {row['mean_time']:.2f}s")
    
    # 4. Recomendaciones basadas en análisis
    print(f"\n4. 💡 RECOMENDACIONES:")
    
    print(f"   🎯 Para MEJOR CALIDAD DE SOLUCIÓN:")
    best_quality_config = config_stats.iloc[0]
    print(f"   - Usar: colony_size={best_quality_config['colony_size']}, "
          f"alpha={best_quality_config['alpha']}, beta={best_quality_config['beta']}, "
          f"q0={best_quality_config['q0']}")
    
    # Buscar configuración con mejor balance calidad/tiempo
    config_stats['efficiency'] = config_stats['mean_cost'] / config_stats['mean_time']
    best_efficiency_idx = config_stats['efficiency'].idxmin()
    best_efficiency_config = config_stats.iloc[best_efficiency_idx]
    
    print(f"\n   ⚡ Para MEJOR BALANCE CALIDAD/TIEMPO:")
    print(f"   - Usar: colony_size={best_efficiency_config['colony_size']}, "
          f"alpha={best_efficiency_config['alpha']}, beta={best_efficiency_config['beta']}, "
          f"q0={best_efficiency_config['q0']}")
    
    # 5. Configuración conservadora (valores seguros)
    print(f"\n   🛡️ CONFIGURACIÓN CONSERVADORA (valores seguros):")
    print(f"   - colony_size: {best_params['colony_size']} (mejor individual)")
    print(f"   - alpha: {best_params['alpha']} (mejor individual)")
    print(f"   - beta: {best_params['beta']} (mejor individual)")
    print(f"   - q0: {best_params['q0']} (mejor individual)")
    
    return {
        'best_individual_params': best_params,
        'best_overall_config': best_run[['colony_size', 'alpha', 'beta', 'q0']].to_dict(),
        'top_5_configs': config_stats,
        'best_quality_config': best_quality_config,
        'best_efficiency_config': best_efficiency_config
    }

def save_analysis_report(df, recommendations, filename=None):
    """
    Guarda un reporte completo del análisis
    """
    if filename is None:
        filename = f"acs_analysis_report_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.txt"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("REPORTE DE ANÁLISIS DE PARÁMETROS - ALGORITMO ACS\n")
        f.write("=" * 60 + "\n\n")
        
        f.write(f"Fecha: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total de experimentos: {len(df)}\n")
        f.write(f"Configuraciones únicas: {len(df.groupby(['colony_size', 'alpha', 'beta', 'q0']))}\n\n")
        
        # Estadísticas generales
        f.write("ESTADÍSTICAS GENERALES:\n")
        f.write(f"- Mejor costo obtenido: {df['final_cost'].min():.2f}\n")
        f.write(f"- Costo promedio: {df['final_cost'].mean():.2f}\n")
        f.write(f"- Desviación estándar: {df['final_cost'].std():.2f}\n")
        f.write(f"- Mejor gap: {df['gap_percentage'].min():.2f}%\n")
        f.write(f"- Gap promedio: {df['gap_percentage'].mean():.2f}%\n\n")
        
        # Mejores parámetros
        f.write("MEJORES CONFIGURACIONES:\n\n")
        f.write("1. Mejor configuración general:\n")
        best_config = recommendations['best_overall_config']
        f.write(f"   - colony_size: {best_config['colony_size']}\n")
        f.write(f"   - alpha: {best_config['alpha']}\n")
        f.write(f"   - beta: {best_config['beta']}\n")
        f.write(f"   - q0: {best_config['q0']}\n\n")
        
        f.write("2. Top 5 configuraciones:\n")
        for i, row in recommendations['top_5_configs'].iterrows():
            f.write(f"   {i+1}. colony_size={row['colony_size']}, alpha={row['alpha']}, "
                   f"beta={row['beta']}, q0={row['q0']} - Costo: {row['mean_cost']:.2f}\n")
    
    print(f"📄 Reporte guardado en: {filename}")
    return filename

# Generar reporte final
if df is not None and param_statistics is not None:
    recommendations = generate_best_parameters_report(df, param_statistics)
    report_file = save_analysis_report(df, recommendations)
    
    print(f"\n✅ ANÁLISIS COMPLETADO")
    print(f"📊 Datos analizados: {len(df)} experimentos")
    print(f"📄 Reporte guardado: {report_file}")
    print(f"🏆 Mejores parámetros identificados y validados estadísticamente")
else:
    print("⚠️  No se puede generar el reporte sin datos completos.")

## 7. Conclusiones y Próximos Pasos

### Conclusiones del Análisis:

1. **Validación de Metodología**: El notebook valida automáticamente si tu metodología de testing es estadísticamente sólida
2. **Análisis Estadístico Robusto**: Incluye intervalos de confianza, pruebas ANOVA y análisis de correlación
3. **Visualizaciones Comprehensivas**: Gráficos que permiten identificar patrones y tendencias fácilmente
4. **Recomendaciones Basadas en Evidencia**: Identificación de los mejores parámetros con justificación estadística

### Para Mejorar tus Tests Actuales:

1. **Ejecuta múltiples corridas** (mínimo 5-10) por cada configuración de parámetros
2. **Usa semillas diferentes** para cada corrida para garantizar independencia estadística
3. **Almacena todos los resultados** en lugar de solo imprimirlos
4. **Mide múltiples métricas**: costo, tiempo de ejecución, convergencia, etc.
5. **Usa condiciones de parada consistentes** (solo por iteraciones, no por costo target)

### Próximos Pasos:

1. Ejecuta el script `parameter_analysis_test.py` mejorado para generar datos
2. Usa este notebook para analizar los resultados
3. Considera análisis adicionales como:
   - Análisis de superficie de respuesta para interacciones entre parámetros
   - Análisis de convergencia temporal
   - Comparación con otros algoritmos metaheurísticos